# Práctica 1

Gonzalo Monedero - Marcos Molina - Nicolás Salgado - Iñigo Vázquez

## Preparación del entorno

In [114]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as ker
import tensorflow as tf
import random

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

# Quitar warnings feos
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Preprocesamiento de datos

### Creación del dataset


In [115]:
iris_RAW = pd.read_csv('../Data/iris.data', sep=',')
iris_OG = iris_RAW.copy()
salidas = iris_RAW.pop("class")


## Creación entradas y salidas

In [116]:
entradas = iris_RAW.to_numpy()

# Categorizamos salidas
label_encoder = LabelEncoder()
salidas = label_encoder.fit_transform(salidas)


# Funciones algoritmo genético

### Variables a definir

In [117]:
"""
ngenes: nneuronas de cada individuo. Sacado a traves del L4P1-EstudioRed.ipynb
NPOB: Número de individuos de la población 
NGEN: Número de generaciones (ciclo completo sobre todos los individuos) 
Pm: Probabilidad de mutación: Probabilidad de que un gen dado mute 
NRES: Cada cuántas generaciones se imprime un resumen de la evolución del proceso 
NSAMPLE: Cada qué número de generaciones se saca un muestreo de la población 
"""

ngenes = 4 # NO TOCAR
npob = 15 # NO TOCAR
ngeneraciones = 1 # TOCAR
pm = 0.1 # TOCAR
nres = 1 # TOCAR
nsample = 1 # TOCAR

### Crear inidividuo

Como bien se dice en el enunciado, un individuo es un MLP sin entrenar, con tantos genes como hemos calculado. En nuestro caso son 7 genes, y el tamaño de entrada son 4. De esta manera creamos un MLP básico.

In [118]:
def crearMLP(ngenesMLP, tam_entrada):
    MLP = ker.Sequential() # Creamos el modelo vacío

    # Añadimos la primera capa (tecnicamente dos, la de entrada (con la variable input_dim) y la oculta)
    MLP.add(ker.layers.Dense(ngenesMLP, input_dim = tam_entrada, activation='relu'))

    # Añadimos la capa de salida
    MLP.add(ker.layers.Dense(3,activation='sigmoid')) 

    # Por último tenemos que configurar el modelo antes de entrenarlo con lo dicho en el enunciado
    # Optimizador: Adam
    # Función error: Mean Squared Error
    # Metricas = Binary accuracy porque nos da el número de precisión de la red
    MLP.compile(optimizer='adam',loss='mean_squared_error',metrics=["accuracy"])
    return MLP

### Crear población

Al igual que antes hemos creado un individuo en este caso crearemos una población. Una población es un conjunto de individuos, en nuestro caso entonces realizaremos un array de población

In [119]:
def crearPoblacion(tamPoblacion,ngenesMLP, tam_entrada):
    poblacion = []
    for i in range (tamPoblacion):
        poblacion.append(crearMLP(ngenesMLP,tam_entrada))
    return poblacion

### Evaluar población

El calculo del fitness de la población se realiza mediante el evaluate del MLP. De esta manera, simplemente tendremos que iterar toda la población y devolver un array con el fitness de cada individuo en la posición de la población.

In [120]:
def evaluarPoblacion(poblacion,entradas_test,esperadas_test):
    evalPobl = []
    for individuo in poblacion:
        metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
        evalPobl.append(metricas_totales*150)
    return evalPobl

### Selección de replicar individuo

Posteriormente necesitaremos un algoritmo de montecarlo, que nos permita elegir la probablididad de selección de el individuo. El método de montecarlo es generar un número aleatorio entre [0,1], si este número es menor que la probabilidad de selección, el individuo se replica, si no, no. En nuestro caso, recibimos una población y la recorremos, viendo cuales se cogen o no. De esta manera, devolvemos un array de booleanos donde indicamos si ese individuo se replica o no.

In [121]:
def seleccion(poblacion,entradas_test,esperadas_test):
    seleccionados = []
    fis_totales = evaluarPoblacion(poblacion,entradas_test,esperadas_test)
    for fi in fis_totales:
        nAleatorio = random.random()
        ps = fi / np.amax(fis_totales)
        if(nAleatorio < ps):
            seleccionados.append(True)
        else:
            seleccionados.append(False)
    return seleccionados

### Mutar

Secuencialmente para todos los genes del padre decide si cada gen se copia fielmente en el hijo o se muta usando un Método de Montecarlo simple sobre el valor de Pm. Si muta, cámbialo por otro valor aleatorio en el rango (-1, 1). Prueba también otras opciones de mutación. 

In [122]:
def mutar(individuo):
    pesos_totales = individuo.get_weights()
    pesos_entrada_oculta = pesos_totales[0]
    pesos_oculta_salida = pesos_totales[2]

    for vector in pesos_entrada_oculta:
        for i in range(len(vector)):
            nAl = random.random()
            if(nAl < pm):
                vector[i] = random.uniform(-1, 1)

    for vector in pesos_oculta_salida:
        for i in range(len(vector)):
            nAl = random.random()
            if(nAl < pm):
                vector[i] = random.uniform(-1, 1)
    
    
    
    return pesos_entrada_oculta,pesos_oculta_salida

### Simulación

In [123]:
def simular():
    poblacion = crearPoblacion(npob,ngenes, 4)
    for i in range(ngeneraciones):
        seleccionados = seleccion(poblacion,entradas,salidas)
simular()

[True, False, False, True, False, False, True, True, True, False, False, False, True, False, True]
